<a href="https://colab.research.google.com/github/msotskaya/compling/blob/main/task2_basic_sotskaya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задача 2: Разметка датасета с HF Datasets

# Критерии оценивания:

1. Выделены все части речи (2 балла)
2. Выделены все именованные сущности (2 балла)
3. Выделены все синтаксические зависимости (2 балла)
4. Результат оформлен в виде датафрейма (2 балла)
5. Датасет опубликован на HF Datasets (2 балла)

**В комментариях к блокноту нужно обязательно добавить ссылку на ваш датасет**

**Шаг 1**: Установка необходимых библиотек и загрузка текста

In [ ]:
# Устанавливаем необходимые библиотеки
!pip install datasets nltk spacy -q

In [ ]:
# Импорт наших инструментов
from datasets import Dataset
import pandas as pd
import spacy
import nltk
from nltk import pos_tag, sent_tokenize, word_tokenize, download
from spacy import displacy

In [ ]:
# Загружаем Spacy
!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Подключаем ресурсы NLTK
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

**Задание**: загрузите текст для разметки

**Опции**:
1. использовать свой текст
2. использовать текст из задания 1: `https://github.com/vifirsanova/compling/blob/main/tasks/task1/data.txt`

In [ ]:
### ваш код здесь ###
!wget https://raw.githubusercontent.com/vifirsanova/compling/refs/heads/main/tasks/task1/data.txt

**Шаг 2**: Выделение частей речи с использованием NLTK

In [ ]:
# Дана функция для выделения частей речи с помощью NLTK
def nltk_pos_tagging(text):
    tokens = word_tokenize(text)
    return pos_tag(tokens)

**Задание**: самостоятельно найдите документацию NLTK и вставьте в свое решение ссылки на описание работы с инструментами `word_tokenize` и `pos_tag` из официальной документации NLTK

Ссылки:

1. https://www.nltk.org/api/nltk.tokenize.html#nltk.tokenize.word_tokenize
2. https://www.nltk.org/api/nltk.tag.html#nltk.tag.pos_tag

**Задание**: произведите частеречную разметку ваших данных с помощью функции `nltk_pos_tagging`

In [ ]:
with open('data.txt', 'r', encoding='UTF-8') as file:
  text = file.read()
texts = sent_tokenize(text)
tagged_words = [nltk_pos_tagging(text) for text in texts]
tagged_words

In [ ]:
"""

ОБРАЗЕЦ ОЖИДАЕМОГО РЕЗУЛЬТАТА ВЫДАЧИ:

- ПРОВЕРЬТЕ ВАШ РЕЗУЛЬТАТ,
- СРАВНИТЕ ЕГО С ОБРАЗЦОМ ПЕРЕД СДАЧЕЙ РАБОТЫ

"""

**Шаг 3**: разметка с помощью spacy

Переключимся на другую бибилиотеку!

**Задание**: допишите функцию для разметки частей речи, именованных сущностей и синтаксическх зависимостей

In [ ]:
def spacy_annotation(text):
    doc = nlp(text) ## примените метод nlp, чтобы произвести разбор текста ##

    ## напишите цикл для выделения частей речи с помощью spacy ##
    ## в качестве подсказки используйте наши практические работы ##
    ## вы должны создать список пар: (токен, частеречный тег) ##
    pos_tags = [(token.text, token.pos_) for token in doc]## создаем список пар: (токена token.text, частеречный тег token.pos_) ##

    ## напишите цикл для выделения именованных сущностей с помощью spacy ##
    ## в качестве подсказки используйте наши практические работы ##
    ## вы должны создать список пар: (текст, тег именованной сущности) ##
    named_entities = [(ent.text, ent.label_) for ent in doc.ents] ## создаем список пар: (токена ent.text, частеречный тег ent.label_) для всех сущностей в списке doc.ents ##

    # как происходит выделение синтаксических зависимостей в spacy?
    # каждый токен связывается с его "головой" (head), что формирует синтаксическую структуру.
    # например, "studying" (основной глагол) может быть связан с "John" как субъект (nsubj)

    dependencies = [(token.text, token.dep_, token.head.text) for token in doc]

    # сформируем словарь, где соберем все результаты нашей разметки
    return {
        "pos_tags": pos_tags,
        "named_entities": named_entities,
        "dependencies": dependencies
        ## создайте здесь словарь с тремя ключами "pos_tags", "named_entities" и "dependencies" ##
        ## запишите в него результаты вашей разметки ##
    }

In [ ]:
# Выполнеяем разметку текста:
# создайте цикл для разметки вашего текста с помощью функции spacy_annotation
spacy_results = [spacy_annotation(text) for text in texts]## здесь будет результат вашей разметки ##

In [ ]:
spacy_results

In [ ]:
"""

ОБРАЗЕЦ ОЖИДАЕМОГО РЕЗУЛЬТАТА ВЫДАЧИ:

- ПРОВЕРЬТЕ ВАШ РЕЗУЛЬТАТ,
- СРАВНИТЕ ЕГО С ОБРАЗЦОМ ПЕРЕД СДАЧЕЙ РАБОТЫ

"""

**Задание**: найдите документацию spacy, добавьте сюда ссылку на объяснение работы token.text, token.dep_.

Ссылка:

1. https://spacy.io/api/dependencyparser#assigned-attributes

**Шаг 4**: Преобразование результатов в датафрейм

In [ ]:
data = []## создайте пустой список ##
import pandas as pd
# начинаем итерацию по данным
# используем enumerate для итерирования по индексам
for i, text in enumerate(texts):
    # добавляем в data все, что мы разметили ранее
    data.append({
        "text": texts[i], ## переменная, в которой хранится ваш текст ##
        "nltk_pos": tagged_words[i],               # Части речи с NLTK
        "spacy_pos": spacy_results[i]["pos_tags"],     # Части речи с SpaCy
        "named_entities": spacy_results[i]["named_entities"],  # Именованные сущности
        "dependencies": spacy_results[i]["dependencies"]  # Синтаксические зависимости
    })

df = pd.DataFrame(data)## создайте датафрейм с data ##

In [ ]:
"""

ОБРАЗЕЦ ОЖИДАЕМОГО РЕЗУЛЬТАТА ВЫДАЧИ:

- ПРОВЕРЬТЕ ВАШ РЕЗУЛЬТАТ,
- СРАВНИТЕ ЕГО С ОБРАЗЦОМ ПЕРЕД СДАЧЕЙ РАБОТЫ

"""

**Шаг 5**: визуализация результатов

In [ ]:
# этот код позволяет отобразить дерево зависимостей с помощью SpaCy
# посмотрите, как работает ваша разметка
# рекомендуется самостоятельное знакомство с библиотекой displacy
example_text = texts[3] # попробуйте визуализировать любой другой текст
doc = nlp(example_text)
displacy.render(doc, style="dep", jupyter=True, options={"distance": 120})

**Шаг 7**: загрузка на HF Datasets

Ссылка на ваш датасет:

1.

In [ ]:
# Создаем датасет
from datasets import Dataset
dataset = Dataset.from_pandas(df)

# Публикация датасета
# Замените 'your_dataset_name' на имя вашего датасета и добавьте токен.
dataset.push_to_hub("dataset_with_text", token="token")

# просмотр результатов
dataset